In [8]:
import os
import numpy as np
import pandas as pd
import feather
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [11]:
top_wd = os.getcwd()
feather_dir = top_wd + "\\Feather"
feather_dir

'c:\\Users\\Andrew\\Documents\\Uni Trier\\Semester 3\\Case Study\\ML Algo\\Feather\\Feather'

In [48]:
os.chdir(feather_dir)
AMELIA = feather.read_dataframe("01_SRS_Base_sample.feather")

,Person_Income,index,AGE,Work_Status,City.Community,District,Current_Education_Activity,Highest_ISCED,Martial_Status,Personal_ID,Province,Regional_ID,Self_Employment_Dummy,Sex,Managerial_Position
1,20691.730613,3331911,33.0,1.0,559,15,2.0,2.0,2.0,126158001.0,4,2,1.0,Male,2.0
2,8115.743923,8514643,40.0,1.0,322,10,2.0,3.0,2.0,321345701.0,3,1,2.0,Female,2.0
3,89558.995107,1285340,64.0,1.0,347,11,2.0,3.0,2.0,48614301.0,4,2,2.0,Male,2.0
4,4965.253598,5716142,43.0,1.0,255,9,2.0,3.0,2.0,216071102.0,3,1,2.0,Male,1.0
5,13338.855808,9469441,64.0,4.0,717,19,2.0,3.0,2.0,357412802.0,5,2,2.0,Female,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1796.947708,3471497,30.0,4.0,1494,38,2.0,4.0,2.0,131441601.0,10,4,2.0,Female,2.0
96,6435.472376,4009577,59.0,1.0,1426,37,2.0,1.0,2.0,151659102.0,10,4,2.0,Female,1.0
97,12507.259084,8269754,25.0,4.0,640,17,2.0,5.0,2.0,312207402.0,5,2,2.0,Female,2.0
98,7328.371205,74883,18.0,2.0,1505,38,2.0,5.0,1.0,2903603.0,10,4,2.0,Male,2.0


In [50]:
def cata_encode(dataframe):
    """Since get_dummies has not dropped the extra column, it must be done explicitly (Avoids Multicolinearity)"""
    #Cities will be dropped since there are too many and they are not considered relevant for our Analysis
    #4: Other inactive person 
    Work_Columns = {"Work_Status_1.0":"At Work", "Work_Status_2.0":"Unemployed", "Work_Status_3.0":"Retired"}
    #5: ISCED5 or ISCED6
    Highest_ISCED = {"Highest_ISCED_1.0":"ISCED 1", "Highest_ISCED_2.0":"ISCED 2", "Highest_ISCED_3.0":"ISCED 3", "Highest_ISCED_4.0":"ISCED 4"}
    #5: Divorced
    Martial_Status = {"Martial_Status_1.0":"Never Married", "Martial_Status_2.0":"Married","Martial_Status_3.0":"Separated","Martial_Status_4.0":"Widowed"}
    #4: Region 4
    Region_ID = {"Regional_ID_1":"Region_1", "Regional_ID_2":"Region_2", "Regional_ID_3":"Region_3"}
    #11: Province 11
    #40: District 40

    dataframe = pd.get_dummies(dataframe, columns=['Work_Status', "Highest_ISCED", "Martial_Status", "Regional_ID", "Province", "District"]).rename(columns=Work_Columns).rename(columns=Highest_ISCED).rename(columns=Martial_Status).rename(columns=Region_ID).drop(columns=["Work_Status_4.0", "Highest_ISCED_5.0", "Martial_Status_5.0", "City.Community", "Regional_ID_4", "Province_11", "District_40"])
    return dataframe

In [51]:
def to_numeric(dataframe):
    sex = {"Male":0,"Female":1}
    dataframe = dataframe.replace({"Sex": sex})
    return dataframe
AMELIA = to_numeric(AMELIA)
AMELIA = cata_encode(AMELIA)

In [52]:
y_var = AMELIA["Person_Income"]
x_var = AMELIA.drop(columns = ["index", "Person_Income", "Personal_ID"])

x_train, x_test, y_train, y_test = train_test_split(x_var, y_var, test_size = 0.25, random_state = 420)

In [53]:
sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)
x_train = pd.DataFrame(x_train, columns=x_var.columns)
x_test = pd.DataFrame(x_test, columns=x_var.columns)
y_train = y_train

In [54]:
param_grid = [
        {
            'hidden_layer_sizes': [(100,), (500,),(1000,),(2000,)],
            'activation' : ['relu'],
            'solver' : ['lbfgs'],
            'alpha': [0.0001, 0.001, 0.01],
            'batch_size':['auto'],
            'max_iter':[500],
            'random_state':[420],
            'verbose':[True]
        }
       ]

In [55]:
clf = GridSearchCV(MLPRegressor(), param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs = -1)
clf.fit(x_train,y_train.ravel())


print("Best parameters set found on development set:")
print(clf.best_params_)

Best parameters set found on development set:
{'activation': 'relu', 'alpha': 0.01, 'batch_size': 'auto', 'hidden_layer_sizes': (2000,), 'max_iter': 500, 'random_state': 420, 'solver': 'lbfgs', 'verbose': True}


C:\Users\Andrew\miniconda3\envs\amelia\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
